In [1]:
import pandas as pd
import numpy as np
import pprint
import operator
import re

pd.options.display.max_columns = None

In [2]:
df = pd.read_csv('./jdp_2018.csv')
df.shape

(71530, 15)

In [3]:
df.head()

,ID,Make,Series,Model and Type,Total Problems,Problems Indicated,Category,Problem,Mode,Verbatim,Mileage,Took to Dealer,Resolved,Severity,Verbatim (Translated)
0,3582A383,Lexus,ES,Lexus ES 4dr Sdn,2,"INT23, INT34",Interior,INT23: Center Console Storage - DTU,D,Storage bin is located too far back in the con...,1000.0,No,NaN,1.0,NaN
1,3582A383,Lexus,ES,Lexus ES 4dr Sdn,2,"INT23, INT34",Interior,INT34: Cup Holders - DTU,D,Cup holders surround the controls on the cente...,1000.0,No,NaN,1.0,NaN
2,358BB95A,Honda,CR-V,Honda CR-V 4dr SUV,1,AEN04,Audio/Communication/Entertainment/Navigation (...,AEN04: Audio System - DTU/Poor Location,D,"There is no turning knob on the radio, only ar...",NaN,No,NaN,3.0,NaN
3,35824B34,Mercedes-Benz,GLA,Mercedes-Benz GLA 4dr SUV,4,"FCD04, AEN03, SEAT17, HVAC02",Features/Controls/Displays (FCD),FCD04: Wipers/Washers - Controls DTU/Poor Loca...,D,Uncomfortable position to turn the back wipers on,NaN,No,NaN,2.0,NaN
4,35824B34,Mercedes-Benz,GLA,Mercedes-Benz GLA 4dr SUV,4,"FCD04, AEN03, SEAT17, HVAC02",Audio/Communication/Entertainment/Navigation (...,AEN03: Audio System - Broken/NW,M,(Copied from AEN97): Audio disconnected by its...,NaN,Yes,Entirely,1.0,NaN


In [4]:
bmw_df = df[df['Make']=='BMW']
mini_df = df[df['Make']=='MINI']

In [5]:
new_df = pd.concat([bmw_df,mini_df],ignore_index=True)
new_df = new_df.fillna("")
new_df['Verbatim'] = new_df['Verbatim'].replace(to_replace='\(([^)]+)\)',value='',regex=True)
print(new_df.shape)
new_df.head()

(3832, 15)


,ID,Make,Series,Model and Type,Total Problems,Problems Indicated,Category,Problem,Mode,Verbatim,Mileage,Took to Dealer,Resolved,Severity,Verbatim (Translated)
0,3579646Q,BMW,X4,BMW X4 4dr SUV,3,"EXT19, FCD31, HVAC08",Exterior,EXT19: Trunk/Hatch/Tailgate - Handle/Latch/Rel...,D,Sweeping door under rear of car rarely opens h...,2550,Yes,Not at all,2,
1,3579646Q,BMW,X4,BMW X4 4dr SUV,3,"EXT19, FCD31, HVAC08",Features/Controls/Displays (FCD),FCD31: Gauges/Instruments - Controls/Displays ...,D,Options to use and enlighten information too h...,2550,No,,4,
2,3579646Q,BMW,X4,BMW X4 4dr SUV,3,"EXT19, FCD31, HVAC08","Heating, Ventilation, and Air Conditioning (HVAC)",HVAC08: Air Conditioning Doesn't Achieve/Maint...,D,Set for 70 in summer and winter and fan blower...,2550,No,,4,
3,35794532,BMW,X1,BMW X1 4dr SUV,1,FCD14,Features/Controls/Displays (FCD),FCD14: Door Locks/Keyless Entry - Controls/Ico...,D,When I enable the security system and my iPhon...,2200,No,,1,
4,35952632,BMW,3 Series,BMW 3 Series 4dr Sdn,3,"EXT01, FCD14, AEN06",Exterior,EXT01: Front Doors - Hard to Open/Close,M,,5000,No,,1,


In [6]:
import nltk
nltk.download('subjectivity')
nltk.download('vader_lexicon')
from nltk.classify import NaiveBayesClassifier
from nltk.corpus import subjectivity
from nltk.sentiment import SentimentAnalyzer
from nltk.sentiment.util import *

[nltk_data] Downloading package subjectivity to
[nltk_data]     /home/qtc5712/nltk_data...
[nltk_data]   Package subjectivity is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/qtc5712/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


/projects/qtb2630/NA_Data_Science/envs/py3/lib/python3.5/site-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [7]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [8]:
verbatim = new_df['Verbatim'].tolist()

In [9]:
sid = SentimentIntensityAnalyzer()

In [10]:
i = 0
makes = []
series = []
problems = []
categories = []
modes = []
statements = []
scores = []

for entry in verbatim:
    #print("{0}".format(new_df.loc[i,'Model and Type']))
    #print("Total Problems: {0}, Category: {1}, Mode: {2}".format(new_df.loc[i,'Total Problems'],new_df.loc[i,'Category'],new_df.loc[i,'Mode']))
    #print(entry)
    ss = sid.polarity_scores(entry)
    j = 0
    for k in sorted(ss):
        #print('{0}: {1}, '.format(k,ss[k]),end='')
        if j == 0:
            scores.append(ss[k])
            j+=1
    #print("\n")
    
    makes.append(new_df.loc[i,'Make'])
    series.append(new_df.loc[i,'Series'])
    problems.append(new_df.loc[i,'Total Problems'])
    categories.append(new_df.loc[i,'Category'])
    modes.append(new_df.loc[i,'Mode'])
    statements.append(entry)
    
    i+=1

In [11]:
stuff = [('Make',makes), ('Series',series), ('Total_Problems',problems), ('Category',categories), ('Mode',modes), ('Verbatim',statements), ('Score',scores)]
new_new_df = pd.DataFrame.from_items(stuff)
new_new_df.head()

,Make,Series,Total_Problems,Category,Mode,Verbatim,Score
0,BMW,X4,3,Exterior,D,Sweeping door under rear of car rarely opens h...,0.0000
1,BMW,X4,3,Features/Controls/Displays (FCD),D,Options to use and enlighten information too h...,0.1100
2,BMW,X4,3,"Heating, Ventilation, and Air Conditioning (HVAC)",D,Set for 70 in summer and winter and fan blower...,0.6124
3,BMW,X1,1,Features/Controls/Displays (FCD),D,When I enable the security system and my iPhon...,0.4939
4,BMW,3 Series,3,Exterior,M,,0.0000


In [12]:
new_new_df.to_csv('2018_sentiments.csv')